In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
mkdir /content/drive/MyDrive/de-reject

mkdir: cannot create directory ‘/content/drive/MyDrive/de-reject’: File exists


In [ ]:
cd /content/drive/MyDrive/de-reject/Data

/content/drive/MyDrive/de-reject/Data


In [ ]:
!unzip *.zip

Archive:  Data.zip
   creating: Data/
  inflating: Data/Assignment_Data.csv  
  inflating: Data/Delivery_Partners_Data.csv  


In [ ]:
ls

In [ ]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [ ]:
!unzip *.zip

Archive:  85561676e53d11eb_2.zip
   creating: dataset/
  inflating: dataset/sample_submission.csv  
  inflating: dataset/train.csv       
  inflating: dataset/test.csv        


In [ ]:
ls

Assignment_Data.csv  Delivery_Partners_Data.csv


In [ ]:
ad = pd.read_csv("Assignment_Data.csv")
de = pd.read_csv("Delivery_Partners_Data.csv")

In [ ]:
df = pd.merge(ad, de, on="DE_ID")

In [ ]:
numeric_columns = df.select_dtypes(include=['number']).columns
numeric_columns = numeric_columns.tolist()

In [ ]:
cat_columns = df.select_dtypes(include=['object']).columns
cat_columns = cat_columns.tolist()

In [ ]:
cat_columns

In [ ]:
numeric_columns

In [ ]:
modes = dict()
means = dict()

In [ ]:
for column in df[numeric_columns]:
    mode = df[column].mean()
    print(mode)
    modes[column] = mode
    # df[column] = df[column].fillna(mode)
    df[column] = df[column].replace(np.nan, mode)

62266.26130338233
615638.894088856
0.060788253244104716
432.8641769398092
364.8571212510735
403.57953834766465
2764.807124351353
24.165407797936464
489.92653685014886
37.38910395510759
72.8590439996829
37.37598328629076
72.88206871033466
484.0220402737284


In [ ]:
for column in df[cat_columns]:
    mode = df[column].mode()[0]
    print(mode)
    means[column] = mode
    df[column] = df[column].replace(np.nan, mode)

2021-03-05 03:43:22.000
2021-03-03 03:06:20.000
Type-1
2021-02-14 05:32:59.753
2021-02-14 07:35:48.000
2021-02-26 03:12:32.000
05:42
2020-08-09


In [ ]:
df.isna().any()

ORDER_ID                    False
DE_ID                       False
ASSIGNMENT_START_TIME       False
ASSIGNMENT_END_TIME         False
reject_ind                  False
reject_type                 False
PLACED_TIME                 False
DELIVERED_TIME              False
LASTMILE_DISTANCE           False
FIRSTMILE_DISTANCE          False
LAST_MILE_TIME_PREDICTED    False
PAYOUT_MADE_TO_DE           False
NUM_PING_COUNT_LAST10MIN    False
LAST_PING_TIME_LAST10MIN    False
CUSTOMER_ZONE               False
CUSTOMER_LAT                False
CUSTOMER_LNG                False
SHIFT_END_TIME              False
DE_HOME_LAT                 False
DE_HOME_LNG                 False
DE_JOINING_DATE             False
DE_ZONE_ID                  False
dtype: bool

In [ ]:
sum(df.isna().any())

0

In [ ]:
df.dtypes

In [ ]:
df.head()

In [ ]:
date_cols = ["DE_JOINING_DATE"]
time_cols = ["SHIFT_END_TIME"]
# datetime_cols_nl = ["ASSIGNMENT_END_TIME", "PLACED_TIME", "DELIVERED_TIME"]
datetime_cols = ["ASSIGNMENT_START_TIME", "LAST_PING_TIME_LAST10MIN", "ASSIGNMENT_END_TIME", "PLACED_TIME", "DELIVERED_TIME"]

In [ ]:
for i in date_cols:
  df[i] = pd.to_datetime(df[i], format = '%Y-%m-%d', errors = 'coerce')

In [ ]:
for i in time_cols:
  df[i] = pd.to_datetime(df[i], format = '%H:%M', errors = 'coerce')

In [ ]:
for i in datetime_cols:
  df[i] = pd.to_datetime(df[i], format = '%Y-%m-%d %H:%M:%S', errors = 'coerce')

In [ ]:
df

In [ ]:
date_cols = ["DE_JOINING_DATE"]
time_cols = ["SHIFT_END_TIME"]
# datetime_cols_nl = ["ASSIGNMENT_END_TIME", "PLACED_TIME", "DELIVERED_TIME"]
datetime_cols = ["ASSIGNMENT_START_TIME", "LAST_PING_TIME_LAST10MIN", "ASSIGNMENT_END_TIME", "PLACED_TIME", "DELIVERED_TIME"]

In [ ]:
for i in date_cols:
  df[i+'_year'] = df[i].dt.year
  df[i+'_month'] = df[i].dt.month
  df[i+'_day'] = df[i].dt.day
  df.pop(i)

In [ ]:
for i in time_cols:
  df[i+'_hour'] = df[i].dt.hour
  df[i+'_minute'] = df[i].dt.minute
  df.pop(i)

In [ ]:
for i in datetime_cols:
  df[i+'_year'] = df[i].dt.year
  df[i+'_month'] = df[i].dt.month
  df[i+'_day'] = df[i].dt.day
  df[i+'_hour'] = df[i].dt.hour
  df[i+'_minute'] = df[i].dt.minute
  df[i+'_second'] = df[i].dt.second
  df.pop(i)

In [ ]:
#Identify categorical and continuous variables
ID_col = ['ORDER_ID']
target_col = ["reject_ind", "reject_type"]

In [ ]:
df

In [ ]:
from sklearn.compose import ColumnTransformer

In [ ]:
cat_cols = ['reject_type']

for var in cat_cols:
    number = LabelEncoder()
    number.fit(df[var])
    df[var] = number.transform(df[var])

In [ ]:
target_col = [2, 3]

In [ ]:
X = df.iloc[:, 1:].values
y = df.iloc[:, target_col].values

In [ ]:
training_portion = .8
train_size = int(len(X) * training_portion)

In [ ]:
train_data = X[0: train_size]
train_labels = y[0: train_size]

validation_data = X[train_size:]
validation_labels = y[train_size:]

In [ ]:
print(train_size)
print(len(train_data))
print(len(train_labels))
print(len(validation_data))
print(len(validation_labels))

105915
105915
105915
26479
26479


# Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import MultiOutputRegressor
model = MultiOutputRegressor(LogisticRegression(multi_class='ovr', max_iter=1000))
model.fit(train_data, train_labels)
y_pred = model.predict(validation_data)

In [ ]:
model.fit(train_data, train_labels)

In [ ]:
model.score(validation_data, validation_labels)

0.5601760622298224

# KNN

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

In [ ]:
knn = KNeighborsRegressor()
knn.fit(train_data, train_labels)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
                    metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                    weights='uniform')

In [ ]:
knn.score(validation_data, validation_labels)

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)


0.516615744112381

# Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeRegressor

In [ ]:
dt = DecisionTreeRegressor()
dt.fit(train_data, train_labels)

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=None,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=None, splitter='best')

In [ ]:
dt.score(validation_data, validation_labels)

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)


1.0

In [ ]:
y_pred_dt = dt.predict(validation_data)

In [ ]:
y_pred_dt

array([[0., 0.],
       [0., 0.],
       [0., 0.],
       ...,
       [1., 1.],
       [1., 0.],
       [1., 2.]])

# Evaluation

In [ ]:
from numpy import absolute
from numpy import mean
from numpy import std
from sklearn.datasets import make_regression
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold

In [ ]:
validation_data validation_labels
train_data train_labels

In [ ]:
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate the model and collect the scores
n_scores = cross_val_score(model, train_data, train_labels, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
# force the scores to be positive
n_scores = absolute(n_scores)
# summarize performance
print('MAE: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))



MAE: 0.023 (0.002)


In [ ]:
n_scores = cross_val_score(model, validation_data, validation_labels, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
# force the scores to be positive
n_scores = absolute(n_scores)
# summarize performance
print('MAE: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

MAE: 0.033 (0.003)


KNN

In [ ]:
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate the model and collect the scores
n_scores = cross_val_score(knn, train_data, train_labels, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
# force the scores to be positive
n_scores = absolute(n_scores)
# summarize performance
print('MAE: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))


MAE: 0.024 (0.001)


In [ ]:
n_scores = cross_val_score(knn, validation_data, validation_labels, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
# force the scores to be positive
n_scores = absolute(n_scores)
# summarize performance
print('MAE: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

MAE: 0.036 (0.002)


DT

In [ ]:
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate the model and collect the scores
n_scores = cross_val_score(dt, train_data, train_labels, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
# force the scores to be positive
n_scores = absolute(n_scores)
# summarize performance
print('MAE: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))


MAE: 0.000 (0.000)


In [ ]:
n_scores = cross_val_score(dt, validation_data, validation_labels, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
# force the scores to be positive
n_scores = absolute(n_scores)
# summarize performance
print('MAE: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

MAE: 0.000 (0.000)


# A Sample Usage of the Models

In [ ]:
ad_test = pd.read_csv("Assignment_Data.csv", nrows=1)
de = pd.read_csv("Delivery_Partners_Data.csv")

In [ ]:
ad_test.pop('DE_ID')

0    180976
Name: DE_ID, dtype: int64

In [ ]:
ad_test

,ORDER_ID,ASSIGNMENT_START_TIME,ASSIGNMENT_END_TIME,reject_ind,reject_type,PLACED_TIME,DELIVERED_TIME,LASTMILE_DISTANCE,FIRSTMILE_DISTANCE,LAST_MILE_TIME_PREDICTED,PAYOUT_MADE_TO_DE,NUM_PING_COUNT_LAST10MIN,LAST_PING_TIME_LAST10MIN,CUSTOMER_ZONE,CUSTOMER_LAT,CUSTOMER_LNG
0,2745,2021-02-13 03:08:24.000,2021-02-13 03:38:26.000,0,NaN,2021-02-13 03:01:11.686,2021-02-13 03:43:26.000,320.9,320.0,314.0,1810.0,63,2021-02-13 03:07:34.000,373.0,37.377569,72.83256


In [ ]:
df.head(1)

,ORDER_ID,DE_ID,reject_ind,reject_type,LASTMILE_DISTANCE,FIRSTMILE_DISTANCE,LAST_MILE_TIME_PREDICTED,PAYOUT_MADE_TO_DE,NUM_PING_COUNT_LAST10MIN,CUSTOMER_ZONE,CUSTOMER_LAT,CUSTOMER_LNG,DE_HOME_LAT,DE_HOME_LNG,DE_ZONE_ID,DE_JOINING_DATE_year,DE_JOINING_DATE_month,DE_JOINING_DATE_day,SHIFT_END_TIME_hour,SHIFT_END_TIME_minute,ASSIGNMENT_START_TIME_year,ASSIGNMENT_START_TIME_month,ASSIGNMENT_START_TIME_day,ASSIGNMENT_START_TIME_hour,ASSIGNMENT_START_TIME_minute,ASSIGNMENT_START_TIME_second,LAST_PING_TIME_LAST10MIN_year,LAST_PING_TIME_LAST10MIN_month,LAST_PING_TIME_LAST10MIN_day,LAST_PING_TIME_LAST10MIN_hour,LAST_PING_TIME_LAST10MIN_minute,LAST_PING_TIME_LAST10MIN_second,ASSIGNMENT_END_TIME_year,ASSIGNMENT_END_TIME_month,ASSIGNMENT_END_TIME_day,ASSIGNMENT_END_TIME_hour,ASSIGNMENT_END_TIME_minute,ASSIGNMENT_END_TIME_second,PLACED_TIME_year,PLACED_TIME_month,PLACED_TIME_day,PLACED_TIME_hour,PLACED_TIME_minute,PLACED_TIME_second,DELIVERED_TIME_year,DELIVERED_TIME_month,DELIVERED_TIME_day,DELIVERED_TIME_hour,DELIVERED_TIME_minute,DELIVERED_TIME_second
0,2745,180976,0,0,320.9,320.0,314.0,1810.0,63,373.0,37.377569,72.83256,37.374634,72.871986,373,2019,10,25,5,42,2021,2,13,3,8,24,2021,2,13,3,7,34,2021,2,13,3,38,26,2021,2,13,3,1,11,2021,2,13,3,43,26


In [ ]:
cp = ad_test.assign(key=0).merge(de.assign(key=0), how='left', on = 'key')

In [ ]:
cp.pop('key')

0      0
1      0
2      0
3      0
4      0
      ..
986    0
987    0
988    0
989    0
990    0
Name: key, Length: 991, dtype: int64

In [ ]:
cp

In [ ]:
numeric_columns = cp.select_dtypes(include=['number']).columns
numeric_columns = numeric_columns.tolist()

In [ ]:
cat_columns = cp.select_dtypes(include=['object']).columns
cat_columns = cat_columns.tolist()

In [ ]:
for column in cp[numeric_columns]:
  if column in means:
    cp[column] = cp[column].replace(np.nan, means[column])

In [ ]:
for column in cp[cat_columns]:
  if column in modes:
    mode = modes[column]
    print(mode)
    cp[column] = cp[column].replace(np.nan, mode)

In [ ]:
date_cols = ["DE_JOINING_DATE"]
time_cols = ["SHIFT_END_TIME"]
# datetime_cols_nl = ["ASSIGNMENT_END_TIME", "PLACED_TIME", "DELIVERED_TIME"]
datetime_cols = ["ASSIGNMENT_START_TIME", "LAST_PING_TIME_LAST10MIN", "ASSIGNMENT_END_TIME", "PLACED_TIME", "DELIVERED_TIME"]

In [ ]:
for i in date_cols:
  cp[i] = pd.to_datetime(cp[i], format = '%Y-%m-%d', errors = 'coerce')

In [ ]:
for i in time_cols:
  cp[i] = pd.to_datetime(cp[i], format = '%H:%M', errors = 'coerce')

In [ ]:
for i in datetime_cols:
  cp[i] = pd.to_datetime(cp[i], format = '%Y-%m-%d %H:%M:%S', errors = 'coerce')

In [ ]:
date_cols = ["DE_JOINING_DATE"]
time_cols = ["SHIFT_END_TIME"]
# datetime_cols_nl = ["ASSIGNMENT_END_TIME", "PLACED_TIME", "DELIVERED_TIME"]
datetime_cols = ["ASSIGNMENT_START_TIME", "LAST_PING_TIME_LAST10MIN", "ASSIGNMENT_END_TIME", "PLACED_TIME", "DELIVERED_TIME"]

In [ ]:
for i in date_cols:
  cp[i+'_year'] = cp[i].dt.year
  cp[i+'_month'] = cp[i].dt.month
  cp[i+'_day'] = cp[i].dt.day
  cp.pop(i)

In [ ]:
for i in time_cols:
  cp[i+'_hour'] = cp[i].dt.hour
  cp[i+'_minute'] = cp[i].dt.minute
  cp.pop(i)

In [ ]:
for i in datetime_cols:
  cp[i+'_year'] = cp[i].dt.year
  cp[i+'_month'] = cp[i].dt.month
  cp[i+'_day'] = cp[i].dt.day
  cp[i+'_hour'] = cp[i].dt.hour
  cp[i+'_minute'] = cp[i].dt.minute
  cp[i+'_second'] = cp[i].dt.second
  cp.pop(i)

In [ ]:
#Identify categorical and continuous variables
ID_col = ['ORDER_ID']
target_col = ["reject_ind", "reject_type"]

In [ ]:
cat_cols = ['reject_type']

for var in cat_cols:
    cp[var] = number.transform(cp[var])

Fitting to same format

In [ ]:
cols = list(df.columns.values)

In [ ]:
cols

In [ ]:
cp = cp[cols]

In [ ]:
target_col = [2, 3]
X = cp.iloc[:, 1:].values

In [ ]:
cp.head(1)

,ORDER_ID,DE_ID,reject_ind,reject_type,LASTMILE_DISTANCE,FIRSTMILE_DISTANCE,LAST_MILE_TIME_PREDICTED,PAYOUT_MADE_TO_DE,NUM_PING_COUNT_LAST10MIN,CUSTOMER_ZONE,CUSTOMER_LAT,CUSTOMER_LNG,DE_HOME_LAT,DE_HOME_LNG,DE_ZONE_ID,DE_JOINING_DATE_year,DE_JOINING_DATE_month,DE_JOINING_DATE_day,SHIFT_END_TIME_hour,SHIFT_END_TIME_minute,ASSIGNMENT_START_TIME_year,ASSIGNMENT_START_TIME_month,ASSIGNMENT_START_TIME_day,ASSIGNMENT_START_TIME_hour,ASSIGNMENT_START_TIME_minute,ASSIGNMENT_START_TIME_second,LAST_PING_TIME_LAST10MIN_year,LAST_PING_TIME_LAST10MIN_month,LAST_PING_TIME_LAST10MIN_day,LAST_PING_TIME_LAST10MIN_hour,LAST_PING_TIME_LAST10MIN_minute,LAST_PING_TIME_LAST10MIN_second,ASSIGNMENT_END_TIME_year,ASSIGNMENT_END_TIME_month,ASSIGNMENT_END_TIME_day,ASSIGNMENT_END_TIME_hour,ASSIGNMENT_END_TIME_minute,ASSIGNMENT_END_TIME_second,PLACED_TIME_year,PLACED_TIME_month,PLACED_TIME_day,PLACED_TIME_hour,PLACED_TIME_minute,PLACED_TIME_second,DELIVERED_TIME_year,DELIVERED_TIME_month,DELIVERED_TIME_day,DELIVERED_TIME_hour,DELIVERED_TIME_minute,DELIVERED_TIME_second
0,2745,141533,0,0,320.9,320.0,314.0,1810.0,63,373.0,37.377569,72.83256,37.390427,72.849424,372,2019,9,20,6,42,2021,2,13,3,8,24,2021,2,13,3,7,34,2021,2,13,3,38,26,2021,2,13,3,1,11,2021,2,13,3,43,26


In [ ]:
model.predict(X)

array([[0, 0],
       [0, 0],
       [0, 0],
       ...,
       [0, 0],
       [0, 0],
       [0, 0]])

In [ ]:
knn.predict(X)

array([[0., 0.],
       [0., 0.],
       [0., 0.],
       ...,
       [0., 0.],
       [0., 0.],
       [0., 0.]])

In [ ]:
dt.predict(X)

array([[0., 0.],
       [0., 0.],
       [0., 0.],
       ...,
       [0., 0.],
       [0., 0.],
       [0., 0.]])